# Advanced SQL

After handling basic sql queries through Python, let us look at slightly more advanced SQL queries and their execution through Python's sqlite module. SQL concepts like nested queries, merges, join operations are some of the basic concepts among advanced sql queries. In order to execute queries and practice these concepts, we need to first load data. Most of these concepts involve more than one table, so we will load and work with data among two tables.

## Introduction to Fandango dataset

Fandango is an online ticketing platform for movies. In order to support user's ticket buying decision, Fandango has viewer reviews and ratings for movie listings. This Fandango data set consists of two files - Fandango scrape and Fandango score comparison. The Fandango scrape consists of the list of movies on Fandango and their ratings. The score comparison file consists of the movie name and compares Fandango's movie score with that of other movie review websites like rotten tomatoes and metacritic.

### Exercise

Connect to fandangodb.db and load the two data sets, fandango_scrape and fandango_score_comparison into two tables 'fscrape' and 'fscore' respectively.

In [1]:
import sqlite3
import csv
import pandas as pd

# Connecting to the database
fancon = sqlite3.connect(':memory:')

# Reading data into tables
scrapedf = pd.read_csv('https://raw.githubusercontent.com/colaberry/538data/master/fandango/fandango_scrape.csv')
scoredf = pd.read_csv('https://raw.githubusercontent.com/colaberry/538data/master/fandango/fandango_score_comparison.csv')
scrapedf.to_sql(name='fscrape',con=fancon,if_exists='append',index=False)
scoredf.to_sql(name='fscore',con=fancon,if_exists='append',index=False)

### Solution code

```python
# Run above code
```

In the above step we have loaded two files fandango_score_comparison and fandango_scrape into two dataframes. Let us verify the success of the loading operation by visualizing the dataframes.

### Exercise

Visualize the heads of both data frames created in the step above. Read and understand the nature of the data.

In [ ]:
# Use .head(5) function on the dataframes to visualize first 5 rows of the dataframes.
# Use print function to print both the dataframes in a single step.

### Solution code

```python
print(scrapedf.head(5),scoredf.head(5))
```

Let us understand a bit more about the data, before using SQL queries to analyze it.

### Exercise

Find the total number of rows in both the tables and print them out. You may use the dataframes to count the data.

In [2]:
# Use a simple print statement to print the number of rows
# Use the length method on the dataframes

### Solution code

```python
print("Total number of records in fscrape table=%d and fscore table=%d"%(len(scrapedf),len(scoredf)))
```

## Union, Intersection and Minus

<img src="../../../images/intersect_union_minus.png" style="width: 65vw;"> <br>

Union is a set theory concept which says that it is a collection of all unique elements of two or more sets. When performed among two tables say by specifying column name, it outputs the collection of all unique values that appear in the common column, of either or both tables. Intersection outputs the values that are appear in both tables. Note that when performed on same inputs, intersection will always be a subset of Union. Minus operation results in values/records, featuring in one table but not featuring in the other - i.e. A-B = A-(Intersection(A,B)).

### Exercise

Execute Union and Intersection operations on 'FILM' column of 'fscore' and 'fscrape' tables to understand the data better as well as understand the operations better.
* 1) Execute the SQL query
* 2) Fetch the output and store it in an array
* 3) Print the first five rows and size of the array to show total number of records in output

In [4]:
fancur = fancon.cursor()

# Modify the code below
fancur.execute("SELECT...UNION...")
unionout = fancur.fetchall()
fancur.execute("...INTERSECT...")
intersectout = fancur.fetchall()
fancur.execute("...EXCEPT...")
exceptout = fancur.fetchall()

# 'MINUS' keyword is not supported in sqlite module at present. 'EXCEPT' keyword can be used as a substitute.
# The EXCEPT query returns the same output as MINUS.

### Solution code

```python
fancur.execute("SELECT FILM FROM fscore UNION SELECT FILM FROM fscrape")
unionout = fancur.fetchall()
fancur.execute("SELECT FILM FROM fscore INTERSECT SELECT FILM FROM fscrape")
intersectout = fancur.fetchall()
fancur.execute("SELECT FILM FROM fscrape EXCEPT SELECT FILM FROM fscore")
exceptout = fancur.fetchall()
print(unionout[:5],intersectout[:5],exceptout[:5])
print("total number of rows in union=%d, total number of rows in intersection=%d, total number of rows in minus=%d"%(len(unionout),len(intersectout),len(exceptout)))
```

### Nested query

A Nested query, also referred to as sub-query, is a query within a query. This means that the innermost query is executed first and the output of that query serves as an input to the query which surrounds the embedded query.

From above analysis we can see that fscrape is a bigger list and fscore is a smaller list of movies. Let us retrieve complete Fandango information for all movies that have an IMDB score of more than 7.5

#### Exercise

Retrieve FILM, STARS, RATING, VOTES for all movies that have an IMDB rating greater than 7.5 in the fscore table. Note: Do not use 'JOIN' operation to solve this exercise.

In [5]:
# Modify the code below
fancur.execute('SELECT...IN (SELECT...)')

### Solution code

```python
fancur.execute('''SELECT * FROM fscrape
                WHERE FILM IN (
                SELECT FILM FROM fscore
                WHERE IMDB>7.5)''')
nestout = fancur.fetchall()
print(nestout[:5])
print("total number of rows=%d"%len(nestout))
```

### Types of joins

A join is an operation that is used to combine data from two tables, given that there is a common column in both of them. There are four types of joins:
1) Inner join - returns common records among both the tables
2) Right join - returns all records from right table, and the common records from the left table
3) Left join - returns all records from left table, and the common records from the right table
4) Full join - returns all records when the common record is either in table 1 or table 2.

Lets understand what they actually mean by executing these operations on the two tables.

#### Exercise (a)

Execute inner join operation on the 'fscore' and 'fscrape' tables. Note that FILM is the common column among both tables.

In [6]:
from tabulate import tabulate

fancur.execute('''SELECT...INNER JOIN...ON...''')
templist1 = fancur.fetchall()
headers1 = []
print(tabulate(templist1, headers1))

### Solution code

```python
fancur.execute('''SELECT fscore.FILM,
                  fscore.RottenTomatoes,
                  fscore.Metacritic,
                  fscore.IMDB,
                  fscrape.RATING,
                  fscrape.VOTES
                  FROM fscore
                  INNER JOIN fscrape
                  ON
                  fscore.FILM=fscrape.FILM''')
templist1 = fancur.fetchall()
headers1 = ['FILM','rottentomatoes','metacritic','Imdb','fandangorating','fandangovotes']
print(tabulate(templist1, headers1))
print(len(templist1))
```

Now that we have seen the results of an 'inner join' operation, Let's replicate the same for a 'left join'.

#### Exercise (b)

Execute a left join operation on the 'fscore' and 'fscrape' tables. Note that FILM is the common column among both tables.

In [7]:
fancur.execute()
templist2 = 
headers2 = []

### Solution code

```python
fancur.execute('''SELECT fscore.FILM,
                  fscore.RottenTomatoes,
                  fscore.Metacritic,
                  fscore.IMDB,
                  fscrape.RATING,
                  fscrape.VOTES
                  FROM fscore
                  LEFT JOIN fscrape
                  ON
                  fscore.FILM=fscrape.FILM''')
templist2 = fancur.fetchall()
headers2 = ['FILM','rottentomatoes','metacritic','Imdb','fandangorating','fandangovotes']
print(tabulate(templist2, headers2))
print(len(templist2))
```

From above code we can see that INNER JOIN and LEFT JOIN produce almost the same output. This is because the fscore table is a subset of fscrape table.

As of now, RIGHT OUTER JOIN and FULL OUTER JOIN are not supported by the sqlite module.

### Stored Procedures, Triggers and Materialized Views

Stored procedures, triggers and materialized views are some of the advanced SQL concepts. A stored procedure is a set of instructions/SQL queries which performs a logical action on the database server and is executed on a recurring basis, much like a standard operating procedure. A trigger is a type of stored procedure, which is set to automatically execute when a certain event occurs in the database (Eg. Say a record in a specific table is updated or deleted). A view is the output of a sql query which is presented in the console, but not stored on disk. When this output is stored on disk, in the form of a structure, in order to retrieve it frequently (instead of having to run the query again), it is called a materialized view.

Some additional reading on the above topics:
Stored procedures: https://en.wikipedia.org/wiki/Stored_procedure, https://www.tutorialspoint.com/t_sql/t_sql_stored_procedures.htm

Triggers: https://www.tutorialspoint.com/plsql/plsql_triggers.htm

Materialized views: https://en.wikipedia.org/wiki/Materialized_view, http://www.postgresqltutorial.com/postgresql-materialized-views/

The above concepts are not supported by sqlite module.

In [1]:
# No exercise

### Solution code

```python
# No exercise
```